In [109]:
! pip install "numpy<1.17"

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

  Using cached https://files.pythonhosted.org/packages/1f/c7/198496417c9c2f6226616cff7dedf2115a4f4d0276613bab842ec8ac1e23/numpy-1.16.4-cp27-cp27mu-manylinux1_x86_64.whl


In [110]:
from keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train.shape

(60000, 28, 28)

In [111]:
from tensorflow.keras import layers

class Linear(layers.Layer):
    def __init__(self,units=128, input_dim=128):
        super(Linear,self).__init__()
        w_init=tf.contrib.layers.xavier_initializer()
        self.weights2=tf.Variable(initial_value=w_init(shape=(input_dim,units),
                                                      dtype=tf.float32),trainable=True)
        b_init=tf.contrib.layers.xavier_initializer()
        self.bias2=tf.Variable(initial_value=b_init(shape=(units,),dtype=tf.float32),
                              trainable=True)
    def call(self,inputs):
        return tf.matmul(inputs,self.weights2)+self.bias2

In [112]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
from tensorflow.contrib.layers.python.layers import batch_norm as batch_norm

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

batch_size=5000

tf.reset_default_graph()
graph = tf.Graph()
with graph.as_default():
    #tf.set_random_seed(2)
    
    X = tf.placeholder(dtype=tf.float32)
    X2 = tf.placeholder(dtype=tf.float32)
    Y = tf.placeholder(dtype=tf.float32)
    
    num_hidden=128
    
    linear_layer = Linear(batch_size,num_hidden)


    # Build a hidden layer Left
    W_hidden = tf.Variable(tf.contrib.layers.xavier_initializer()(shape=[784, num_hidden]))
    b_hidden = tf.Variable(tf.contrib.layers.xavier_initializer()(shape=[num_hidden]))
    p_hidden = tf.nn.relu( tf.add(tf.matmul(X, W_hidden), b_hidden) )

    W_hidden2 = tf.Variable(tf.contrib.layers.xavier_initializer()(shape=[num_hidden, num_hidden]))
    b_hidden2 = tf.Variable(tf.contrib.layers.xavier_initializer()(shape=[num_hidden]))
    p_hidden2 = tf.layers.batch_normalization(tf.reshape(tf.add(tf.matmul(p_hidden, W_hidden2), b_hidden2) ,[batch_size,num_hidden]))
    p_hidden22 = tf.nn.relu(p_hidden2)

    # Build a hidden layer Right
    W_hiddenR = tf.Variable(tf.contrib.layers.xavier_initializer()(shape=[784, num_hidden]))
    b_hiddenR = tf.Variable(tf.contrib.layers.xavier_initializer()(shape=[num_hidden]))
    p_hiddenR = tf.nn.relu( tf.add(tf.matmul(X2, W_hiddenR), b_hiddenR) )

    W_hidden2R = tf.Variable(tf.contrib.layers.xavier_initializer()(shape=[num_hidden, num_hidden]))
    b_hidden2R = tf.Variable(tf.contrib.layers.xavier_initializer()(shape=[num_hidden]))
    p_hidden2R = tf.layers.batch_normalization(tf.reshape(tf.add(tf.matmul(p_hiddenR, W_hidden2R), b_hidden2R),[batch_size,num_hidden]))
    p_hidden2R2 = tf.nn.relu(p_hidden2R)

    # Conncatenate Left + Right
    W_concat = tf.Variable(tf.contrib.layers.xavier_initializer()(shape=[num_hidden,num_hidden]))
    b_concat = tf.Variable(tf.zeros(shape=[num_hidden]))
    p_concat2 = tf.add(tf.matmul(tf.divide(tf.math.add(p_hidden22,p_hidden2R2),2), W_concat), b_concat)
    p_concat3 = tf.layers.batch_normalization(tf.reshape(p_concat2,[batch_size,num_hidden]))
    linear1=linear_layer(p_concat3)
    p_concat22=tf.nn.relu(linear1)
    
    # Build the output layer
    W_output = tf.Variable(tf.contrib.layers.xavier_initializer()(shape=[batch_size, 10]))
    b_output = tf.Variable(tf.zeros(shape=[10]))
    p_output = tf.nn.softmax( tf.add(tf.matmul(p_concat22, W_output), b_output))
    
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y,logits=p_output))
    acc, acc_op = tf.metrics.accuracy(labels=tf.argmax(Y,1), 
                                      predictions=tf.argmax(p_output,1))
    
    minimization_op = tf.train.GradientDescentOptimizer(learning_rate=0.001).minimize(loss)
    saver = tf.train.Saver()
    
    limit=10000
    
    def norm(x):
        return (x-np.min(x))/(np.max(x)-np.min(x))

    feed_dict = {
        X: norm(x_train.reshape(-1,784)),
        X2: norm(x_train.reshape(-1,784)),
        Y: pd.get_dummies(y_train)
    }

/home/rubensvectomobile_gmail_com/.local/lib/python3.5/site-packages/tensorflow/python/client/session.py:1735: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [113]:
def next_batch(num, data, labels):
    idx = np.arange(0 , len(data))
    np.random.shuffle(idx)
    idx = idx[:num]
    data_shuffle = [data[ i] for i in idx]
    labels_shuffle = [labels[ i] for i in idx]
    return np.asarray(data_shuffle).astype(np.float32), np.asarray(labels_shuffle).astype(np.float32)

In [ ]:

with tf.Session(graph=graph, config=config) as session:
    #ckpt = "/home/rubensvectomobile/Scratch/Train/train4/model.ckpt"
    #saver.restore(session, ckpt)

    init = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())
    session.run(init)
    
    for step in range(35000):

        X0, Y0 = next_batch(batch_size, x_train, y_train)
        X02=X0
        
        feed_dict = {
        X: norm(X0.reshape(-1,784)),
        X2: norm(X02.reshape(-1,784)),
        Y: pd.get_dummies(Y0)}
        
        J_value = session.run(loss, feed_dict)
        acc = (batch_size-session.run(tf.count_nonzero(tf.add(tf.argmax(session.run(Y, feed_dict),1),-tf.argmax(session.run(p_output, feed_dict),1)))))/batch_size

        session.run(minimization_op, feed_dict)
        if step % 10 == 0:
            print("Step:", step, " Loss:", J_value," Accuracy:", acc)
            if acc > 0.92:
                save_path = saver.save(session, "/home/rubensvectomobile/Scratch/Train/train5/model.ckpt")
                print("Model saved in path: %s" % save_path)



Step: 0  Loss: 2.3025355  Accuracy: 0.1278
Step: 10  Loss: 2.3026452  Accuracy: 0.1302
Step: 20  Loss: 2.3025477  Accuracy: 0.127
Step: 30  Loss: 2.3026419  Accuracy: 0.13
Step: 40  Loss: 2.302527  Accuracy: 0.1328
Step: 50  Loss: 2.3026233  Accuracy: 0.1258
Step: 60  Loss: 2.3025606  Accuracy: 0.1322
Step: 70  Loss: 2.3026102  Accuracy: 0.1244
Step: 80  Loss: 2.302587  Accuracy: 0.133
Step: 90  Loss: 2.3024757  Accuracy: 0.133
Step: 100  Loss: 2.3024933  Accuracy: 0.1294
Step: 110  Loss: 2.30279  Accuracy: 0.1186
Step: 120  Loss: 2.3027296  Accuracy: 0.1168
Step: 130  Loss: 2.3026624  Accuracy: 0.1222
Step: 140  Loss: 2.3025994  Accuracy: 0.1282
Step: 150  Loss: 2.3025682  Accuracy: 0.13
Step: 160  Loss: 2.3026538  Accuracy: 0.128
Step: 170  Loss: 2.302488  Accuracy: 0.1352
Step: 180  Loss: 2.3024745  Accuracy: 0.1346
Step: 190  Loss: 2.3024101  Accuracy: 0.1332
Step: 200  Loss: 2.3026133  Accuracy: 0.1248
Step: 210  Loss: 2.3024814  Accuracy: 0.131
Step: 220  Loss: 2.3026676  Accurac

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, np.argmax(np.array(pred00).reshape(60000,-1),axis=1))